<a href="https://colab.research.google.com/github/JyLee98ImTrying/AA2/blob/main/AA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# Import Dataset & Install Packages
import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')
url = 'https://raw.githubusercontent.com/amankharwal/Website-data/master/marketing_campaign.csv'
data=pd.read_csv((url),header=0,sep=';')

In [54]:
#Define Customer Personalities

#Spending variable creation
data['Age']=2014-data['Year_Birth']

data['Spending']=data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
#Seniority variable creation
last_date = date(2014,10, 4)
data['Seniority']=pd.to_datetime(data['Dt_Customer'], dayfirst=True,format = '%Y-%m-%d')
data['Seniority'] = pd.to_numeric(data['Seniority'].dt.date.apply(lambda x: (last_date - x)).dt.days, downcast='integer')/30
data=data.rename(columns={'NumWebPurchases': "Web",'NumCatalogPurchases':'Catalog','NumStorePurchases':'Store'})
data['Marital_Status']=data['Marital_Status'].replace({'Divorced':'Alone','Single':'Alone','Married':'In couple','Together':'In couple','Absurd':'Alone','Widow':'Alone','YOLO':'Alone'})
data['Education']=data['Education'].replace({'Basic':'Undergraduate','2n Cycle':'Undergraduate','Graduation':'Postgraduate','Master':'Postgraduate','PhD':'Postgraduate'})

data['Children']=data['Kidhome']+data['Teenhome']
data['Has_child'] = np.where(data.Children> 0, 'Has child', 'No child')
data['Children'].replace({3: "3 children",2:'2 children',1:'1 child',0:"No child"},inplace=True)
data=data.rename(columns={'MntWines': "Wines",'MntFruits':'Fruits','MntMeatProducts':'Meat','MntFishProducts':'Fish','MntSweetProducts':'Sweets','MntGoldProds':'Gold'})


data=data[['Age','Education','Marital_Status','Income','Spending','Seniority','Has_child','Children','Wines','Fruits','Meat','Fish','Sweets','Gold']]
data.head()

,Age,Education,Marital_Status,Income,Spending,Seniority,Has_child,Children,Wines,Fruits,Meat,Fish,Sweets,Gold
0,57,Postgraduate,Alone,58138.000,1617,25.333,No child,No child,635,88,546,172,88,88
1,60,Postgraduate,Alone,46344.000,27,7.000,Has child,2 children,11,1,6,2,1,6
2,49,Postgraduate,In couple,71613.000,776,13.633,No child,No child,426,49,127,111,21,42
3,30,Postgraduate,In couple,26646.000,53,7.867,Has child,1 child,11,4,20,10,3,5
4,33,Postgraduate,In couple,58293.000,422,8.600,Has child,1 child,173,43,118,46,27,15


In [60]:
# Check if there are "Alone" in Marital Status but "3 children",'2 children',1:'1 child' in Children
mask = (data['Marital_Status'] == 'Alone') & (data['Children'].isin(["3 children", "2 children", "1 child"]))
filtered_data = data[mask]

# Print the number of rows that match the condition
print("Number of rows matching the condition: ", filtered_data.shape[0])

Number of rows matching the condition:  534


In [55]:
data=data.dropna(subset=['Income'])
data=data[data['Income']<600000]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2215 entries, 0 to 2239
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             2215 non-null   int64  
 1   Education       2215 non-null   object 
 2   Marital_Status  2215 non-null   object 
 3   Income          2215 non-null   float64
 4   Spending        2215 non-null   int64  
 5   Seniority       2215 non-null   float64
 6   Has_child       2215 non-null   object 
 7   Children        2215 non-null   object 
 8   Wines           2215 non-null   int64  
 9   Fruits          2215 non-null   int64  
 10  Meat            2215 non-null   int64  
 11  Fish            2215 non-null   int64  
 12  Sweets          2215 non-null   int64  
 13  Gold            2215 non-null   int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 259.6+ KB


In [59]:
marital_status_counts = data['Marital_Status'].value_counts()
marital_status_percentages = marital_status_counts / marital_status_counts.sum() * 100
print(marital_status_percentages)

#Calculate the percentage of each education group
education_counts = data['Education'].value_counts()
education_percentages = education_counts / education_counts.sum() * 100
print(education_percentages)

#Calculate the percentage of each children group
children_counts = data['Children'].value_counts()
children_percentages = children_counts / children_counts.sum() * 100
print(children_percentages)


In couple   65
Alone       35
Name: Marital_Status, dtype: float64
Postgraduate    89
Undergraduate   11
Name: Education, dtype: float64
1 child      50
No child     29
2 children   19
3 children    2
Name: Children, dtype: float64


In [56]:
data.to_csv(index=False)

'Age,Education,Marital_Status,Income,Spending,Seniority,Has_child,Children,Wines,Fruits,Meat,Fish,Sweets,Gold\n57,Postgraduate,Alone,58138.0,1617,25.333333333333332,No child,No child,635,88,546,172,88,88\n60,Postgraduate,Alone,46344.0,27,7.0,Has child,2 children,11,1,6,2,1,6\n49,Postgraduate,In couple,71613.0,776,13.633333333333333,No child,No child,426,49,127,111,21,42\n30,Postgraduate,In couple,26646.0,53,7.866666666666666,Has child,1 child,11,4,20,10,3,5\n33,Postgraduate,In couple,58293.0,422,8.6,Has child,1 child,173,43,118,46,27,15\n47,Postgraduate,In couple,62513.0,716,13.0,Has child,1 child,520,42,98,0,42,14\n43,Postgraduate,Alone,55635.0,590,23.0,Has child,1 child,235,65,164,50,49,27\n29,Postgraduate,In couple,33454.0,169,17.133333333333333,Has child,1 child,76,10,56,3,1,23\n40,Postgraduate,In couple,30351.0,46,16.166666666666668,Has child,1 child,14,0,24,3,3,2\n64,Postgraduate,In couple,5648.0,49,6.833333333333333,Has child,2 children,28,0,6,1,1,13\n38,Undergraduate,In couple,

In [57]:
#Clustering of Data
scaler=StandardScaler()
dataset_temp=data[['Income','Seniority','Spending']]
X_std=scaler.fit_transform(dataset_temp)
X = normalize(X_std,norm='l2')

gmm=GaussianMixture(n_components=4, covariance_type='spherical',max_iter=2000, random_state=5).fit(X)
labels = gmm.predict(X)
dataset_temp['Cluster'] = labels
dataset_temp=dataset_temp.replace({0:'High potential', 3:'Need Attention', 1:'Stars',2:'Leaky bucket'})
data = data.merge(dataset_temp.Cluster, left_index=True, right_index=True)

pd.options.display.float_format = "{:.0f}".format
summary=data[['Income','Spending','Seniority','Cluster']]
summary.set_index("Cluster", inplace = True)
summary=summary.groupby('Cluster').describe().transpose()
summary.head()

Cluster       High potential  Leaky bucket  Need Attention  Stars
Income count             462           584             641    528
       mean            73438         34757           37705  69542
       std             13753         12075           12397  12006
       min             49090          2447            1730  44802
       25%             65298         26489           28839  60880

In [58]:

#Create Age segment
data['Age_group'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = ['Low income', 'Low to medium income', 'Medium to high income', 'High income']
data['Income_group'] = pd.qcut(data['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = ['New customers', 'Discovering customers', 'Experienced customers', 'Old customers']
data['Seniority_group'] = pd.qcut(data['Seniority'], q=4, labels=cut_labels_Seniority)
data['Income_copy'] = data['Income']
data['Age_copy'] = data['Age']
data=data.drop(columns=['Age','Income','Seniority'])

In [31]:

low_income_avg = data.loc[data['Income_group'] == 'Low income', 'Income_copy'].mean()
low_med_income_avg = data.loc[data['Income_group'] == 'Low to medium income', 'Income_copy'].mean()
med_high_income_avg = data.loc[data['Income_group'] == 'Medium to high income', 'Income_copy'].mean()
high_income_avg = data.loc[data['Income_group'] == 'High income', 'Income_copy'].mean()

#Print the averages
print("Low income average: ", low_income_avg)
print("Low to medium income average: ", low_med_income_avg)
print("Medium to high income average: ", med_high_income_avg)
print("High income average: ", high_income_avg)

Low income average:  25187.106498194946
Low to medium income average:  43060.120938628155
Medium to high income average:  60160.88267148015
High income average:  79521.06690777576


In [61]:
#Calculate average age for each age group
young_avg = data.loc[data['Age_group'] == 'Young', 'Age_copy'].mean()
adult_avg = data.loc[data['Age_group'] == 'Adult', 'Age_copy'].mean()
mature_avg = data.loc[data['Age_group'] == 'Mature', 'Age_copy'].mean()
senior_avg = data.loc[data['Age_group'] == 'Senior', 'Age_copy'].mean()

#Print the averages
print("Young average: ", young_avg)
print("Adult average: ", adult_avg)
print("Mature average: ", mature_avg)
print("Senior average: ", senior_avg)


Young average:  26.392996108949415
Adult average:  38.892116182572614
Mature average:  54.967213114754095
Senior average:  69.1025641025641


In [62]:
#Calculate the percentage of each age group
age_counts = data['Age_group'].value_counts()
age_counts = age_counts.sort_index()
age_percentages = age_counts / age_counts.sum() * 100
print(age_percentages)

Young    12
Adult    44
Mature   41
Senior    4
Name: Age_group, dtype: float64


In [63]:
cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
data['Wines_segment'] = pd.qcut(data['Wines'][data['Wines']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fruits_segment'] = pd.qcut(data['Fruits'][data['Fruits']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Meat_segment'] = pd.qcut(data['Meat'][data['Meat']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fish_segment'] = pd.qcut(data['Fish'][data['Fish']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Sweets_segment'] = pd.qcut(data['Sweets'][data['Sweets']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Gold_segment'] = pd.qcut(data['Gold'][data['Gold']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data.replace(np.nan, "Non consumer",inplace=True)
data.drop(columns=['Spending','Wines','Fruits','Meat','Fish','Sweets','Gold'],inplace=True)
data = data.astype(object)

In [74]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format
association=data.copy() 
df = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product='Gold'
segment='Biggest consumer'
target = '{\'%s_segment_%s\'}' %(product,segment)
results_personnal_care = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results_personnal_care.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
652,(Fish_segment_Biggest consumer),(Gold_segment_Biggest consumer),0.207,0.243,0.108,0.523,2.149,0.058,1.586
2126,"(Fruits_segment_Biggest consumer, Education_Postgraduate)",(Gold_segment_Biggest consumer),0.183,0.243,0.095,0.520,2.136,0.051,1.575
598,(Fruits_segment_Biggest consumer),(Gold_segment_Biggest consumer),0.203,0.243,0.105,0.518,2.128,0.056,1.569
2306,"(Education_Postgraduate, Fish_segment_Biggest consumer)",(Gold_segment_Biggest consumer),0.183,0.243,0.095,0.517,2.126,0.050,1.567
384,(Cluster_Stars),(Gold_segment_Biggest consumer),0.238,0.243,0.123,0.515,2.117,0.065,1.561
